In [1]:
import os
import sys
import pandas as pd

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from functions import get_dates, get_accesstoken, get_months_dates

In [4]:
from talech_keys import grant_type, client_id_1, client_id_2, client_secret_1, client_secret_2, \
    client_version, token_url, refresh_token_1, refresh_token_2, ID_HQ, ID_IFSC, ID_PSL, ID_SAS, ID_TCD

In [388]:
accesstoken = get_accesstoken(client_id_1, client_secret_1, refresh_token_1, client_version, token_url)

In [389]:
token = {
    'securityToken': accesstoken,
    'X-POS-MerchantId': ID_SAS
}

In [390]:
month = 1
year = 2019
days = get_months_dates(0, 1, month, year)

In [391]:
print(days)

['01/01/2019 00:00:00', '01/02/2019 00:00:00', '01/03/2019 00:00:00', '01/04/2019 00:00:00', '01/05/2019 00:00:00', '01/06/2019 00:00:00', '01/07/2019 00:00:00', '01/08/2019 00:00:00', '01/09/2019 00:00:00', '01/10/2019 00:00:00', '01/11/2019 00:00:00', '01/12/2019 00:00:00', '01/13/2019 00:00:00', '01/14/2019 00:00:00', '01/15/2019 00:00:00', '01/16/2019 00:00:00', '01/17/2019 00:00:00', '01/18/2019 00:00:00', '01/19/2019 00:00:00', '01/20/2019 00:00:00', '01/21/2019 00:00:00', '01/22/2019 00:00:00', '01/23/2019 00:00:00', '01/24/2019 00:00:00', '01/25/2019 00:00:00', '01/26/2019 00:00:00', '01/27/2019 00:00:00', '01/28/2019 00:00:00', '01/29/2019 00:00:00', '01/30/2019 00:00:00', '01/31/2019 00:00:00']


In [392]:
import datetime
import requests

In [393]:
def json_get_sales_count(obj):
    """Recursively fetch values from nested JSON."""
    arr = {'bakedgoods': {}, 'breakfast': {}, 'lunch': {}}
    itemvalue = None
    categoryname = None
    itemname = None

    def extract(obj, arr, itemvalue, categoryname, itemname):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, itemvalue, categoryname, itemname)
                elif k == 'item':
                    itemkey = v
                elif k == 'soldQuantity':
                    itemvalue = v
                elif k == 'categoryName':
                    if v == 'LUNCH':
                        categoryname = 'lunch'
                    elif v == 'BREAKFAST':
                        categoryname = 'breakfast'
                    elif v == 'BAKED GOODS':
                        categoryname = 'bakedgoods'
                elif k == 'productName':
                    v = v.replace(' + ', '')
                    v = v.replace('To Go', '')
                    v = v.replace('Sit In', '')
                    v = v.replace('/w Mixed Seeds, Nuts & Honey', '')
                    v = v.replace('/w Honey', '')
                    v = v.replace(', ', '')
                    v = v.replace('Not Heated', '')
                    v = v.replace('Heated', '')
                    v = v.replace('Soup & Sandwich', '')
                    v = v.replace('Ham Toastie', 'Ham & Cheese Toastie')
                    itemname = v

                if categoryname and itemvalue and itemname:
                    if itemname in arr[categoryname]:
                        arr[categoryname][itemname] += itemvalue
                    else:
                        arr[categoryname][itemname] = itemvalue
                    itemvalue = None
                    categoryname = None
                    itemname = None

        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, itemvalue, categoryname, itemname)
        return arr

    values = extract(obj, arr, itemvalue, categoryname, itemname)
    return values

In [394]:
def get_summary_report(token, startdate, enddate):
    arr = {
        'searchCriteria': {
            'startDate': startdate,
            'endDate': enddate,
            'includedReports': [102]
        }
    }
    result = requests.post(url='https://mapi-eu.talech.com/reports/receiptssummaryreport',
                           json=arr,
                           headers=token)
    items = result.json()
    salesdata = json_get_sales_count(items)
    return salesdata
    return items

In [395]:
dailysales = {}
for i in range(0, len(days)):
    d = pd.to_datetime(days[i])
    end = str(d.replace(hour=23).strftime('%m/%d/%Y %H:%M:%S'))
    salesdata = get_summary_report(token, days[i], end)
    dailysales[days[i]] = salesdata

In [396]:
dailysales

{'01/01/2019 00:00:00': {'bakedgoods': {}, 'breakfast': {}, 'lunch': {}},
 '01/02/2019 00:00:00': {'bakedgoods': {'Mixed Berry Scone': 17,
   'Pain au Chocolat': 5,
   'Croissant': 5,
   'Oatmeal & Raisin Cookie': 3,
   'Custard Tart': 4,
   'Cinnabun': 5,
   'Anzac Biscuit': 9,
   'Snickers Boost Ball': 8,
   'Almond Financier': 1,
   'Coconut Snowball': 3,
   'Pain au Raisin': 8,
   'Peanut Butter Brownie': 2,
   'Wholemeal Muffin': 3,
   'Festive Financier': 1},
  'breakfast': {'Sourdough Toast': 4,
   'Sausage Roll': 8,
   'Organic Porridge': 4,
   'Ham Cheese Croissant': 3,
   'Berry & Granola Parfait': 3},
  'lunch': {'Caprese': 2,
   'Turkey Sandwich': 2,
   'Chicken Bacon Wrap': 2,
   'Chilli Tuna Wrap': 1,
   'Ham & Cheese Toastie': 2,
   'Soup of the Day': 2}},
 '01/03/2019 00:00:00': {'bakedgoods': {'Mixed Berry Scone': 18,
   'Cinnabun': 6,
   'Pain au Chocolat': 4,
   'Croissant': 5,
   'Custard Tart': 7,
   'Oatmeal & Raisin Cookie': 2,
   'Festive Financier': 3,
   'Anza

In [397]:
dailysales_df = pd.DataFrame(dailysales)

In [398]:
dailysales_df

01/01/2019 00:00:00  \
bakedgoods                  {}   
breakfast                   {}   
lunch                       {}   

                                          01/02/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 17, 'Pain au Chocolat': ...   
breakfast   {'Sourdough Toast': 4, 'Sausage Roll': 8, 'Org...   
lunch       {'Caprese': 2, 'Turkey Sandwich': 2, 'Chicken ...   

                                          01/03/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 18, 'Cinnabun': 6, 'Pain...   
breakfast   {'Sourdough Toast': 1, 'Sausage Roll': 8, 'Ham...   
lunch       {'Turkey Sandwich': 3, 'Chicken Bacon Wrap': 2...   

                                          01/04/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 19, 'Cinnabun': 6, 'Pain...   
breakfast   {'Sourdough Toast': 1, 'Sausage Roll': 7, 'Org...   
lunch       {'Turkey Sandwich': 1, 'Mediterranean Vegan Wr...   

                                          01/05/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 15, 'Cinnabun': 8, 'Pain...   
breakfast   {'Sausage Roll': 7, 'Ham Cheese Croissant': 4,...   
lunch       {'Mediterranean Vegan Wrap': 2, 'Turkey Sandwi...   

                                          01/06/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 12, 'Cinnabun': 5, 'Croi...   
breakfast      {'Sausage Roll': 9, 'Ham Cheese Croissant': 4}   
lunch                                                      {}   

                                          01/07/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 15, 'Coconut Snowball': ...   
breakfast   {'Sourdough Toast': 2, 'Sausage Roll': 3, 'Org...   
lunch       {'Turkey Sandwich': 2, 'Caprese': 2, 'Chicken ...   

                                          01/08/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 8, 'Cinnabun': 7, 'Oatme...   
breakfast   {'Sourdough Toast': 5, 'Organic Porridge': 5, ...   
lunch       {'Turkey Sandwich': 1, 'Chicken Bacon Wrap': 3...   

                                          01/09/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 15, 'Coconut Snowball': ...   
breakfast   {'Sourdough Toast': 4, 'Organic Porridge': 5, ...   
lunch       {'Turkey Sandwich': 2, 'Ham & Cheese Toastie':...   

                                          01/10/2019 00:00:00  ...  \
bakedgoods  {'Mixed Berry Scone': 21, 'Cinnabun': 6, 'Pain...  ...   
breakfast   {'Sourdough Toast': 6, 'Sausage Roll': 7, 'Org...  ...   
lunch       {'Caprese': 2, 'Mediterranean Vegan Wrap': 2, ...  ...   

                                          01/22/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 18, 'Cinnabun': 6, 'Pain...   
breakfast   {'Sourdough Toast': 3, 'Sausage Roll': 4, 'Org...   
lunch       {'Mediterranean Vegan Wrap': 1, 'Ham & Cheese ...   

                                          01/23/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 22, 'Cinnabun': 7, 'Pain...   
breakfast   {'Sourdough Toast': 3, 'Sausage Roll': 4, 'Org...   
lunch       {'Mediterranean Vegan Wrap': 1, 'Chicken Bacon...   

                                          01/24/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 25, 'Cinnabun': 8, 'Oatm...   
breakfast   {'Sourdough Toast': 6, 'Sausage Roll': 9, 'Org...   
lunch       {'Caprese': 1, 'Ham & Cheese Toastie': 2, 'Tur...   

                                          01/25/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 31, 'Cinnabun': 9, 'Pain...   
breakfast   {'Sourdough Toast': 3, 'Sausage Roll': 7, 'Org...   
lunch       {'Turkey Sandwich': 2, 'Ham & Cheese Toastie':...   

                                          01/26/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 9, 'Cinnabun': 7, 'Pain ...   
breakfast   {'Sausage Roll': 2, 'Ham Cheese Croissant': 2,...   
lunch       {'Turkey Sandwich': 2, 'Ham & Cheese Toastie':...   

                                          01/27/2019 00:00:00  \
bakedgoods  {'Mixed Berry Scone': 12, 'Cinnabun': 5, 'Pain...   
breakfast   {'Sausage Roll': 4, 'Ham Cheese Croissant': 3,...   
lunch

In [399]:
dailysales_df.loc['bakedgoods', '01/02/2019 00:00:00']

{'Mixed Berry Scone': 17,
 'Pain au Chocolat': 5,
 'Croissant': 5,
 'Oatmeal & Raisin Cookie': 3,
 'Custard Tart': 4,
 'Cinnabun': 5,
 'Anzac Biscuit': 9,
 'Snickers Boost Ball': 8,
 'Almond Financier': 1,
 'Coconut Snowball': 3,
 'Pain au Raisin': 8,
 'Peanut Butter Brownie': 2,
 'Wholemeal Muffin': 3,
 'Festive Financier': 1}

In [400]:
from dataclasses import dataclass

In [401]:
class SalesGrowth:
    def __init__(self, category, day, percentile):
        self.category = category
        self.day = day
        self.percentile = percentile
        
    def __iter__(self):
        yield self.category
        yield self.day
        yield self.percentile
        
    @classmethod
    def columns(cls):
        return['category', 'day', 'percentile']

In [402]:
class AllSalesGrowth:
    def __init__(self):
        self.lst = []
        
    def set_item(self, item):
        self.lst.append(item)
        
    def get_item(self, category, day):
        for i in range(len(self.lst)):
            if self.lst[i].category == category and self.lst[i].day == day:
                return self.lst[i]
        return False
    
    def get_percentile(self, category, day):
        for i in range(len(self.lst)):
            if self.lst[i].category == category and self.lst[i].day == day:
                    return self.lst[i].percentile
                
    @property
    def items(self):
        return pd.DataFrame(self.lst, columns=SalesGrowth.columns())
                

In [403]:
import datetime

In [404]:
growth_json = pd.read_json('data/percentile_change/{}_{}.json'.format(month, year))
growth_json

0     1     2     3     4     5     6
bakedgoods  14.3  12.4   8.6   6.0   1.5  -2.0  24.0
breakfast   50.0  41.3  48.8  45.8  47.2  34.3  43.9
lunch       42.1  46.4  46.4  43.0  46.0  23.4 -16.1

In [405]:
growths = AllSalesGrowth()

for category in growth_json.index:
    for day in growth_json.columns:    
        percentile = growth_json.loc[category, day]
        growth = SalesGrowth(category, day, percentile)
        growths.set_item(growth)

In [406]:
growths.items

category  day  percentile
0   bakedgoods    0        14.3
1   bakedgoods    1        12.4
2   bakedgoods    2         8.6
3   bakedgoods    3         6.0
4   bakedgoods    4         1.5
5   bakedgoods    5        -2.0
6   bakedgoods    6        24.0
7    breakfast    0        50.0
8    breakfast    1        41.3
9    breakfast    2        48.8
10   breakfast    3        45.8
11   breakfast    4        47.2
12   breakfast    5        34.3
13   breakfast    6        43.9
14       lunch    0        42.1
15       lunch    1        46.4
16       lunch    2        46.4
17       lunch    3        43.0
18       lunch    4        46.0
19       lunch    5        23.4
20       lunch    6       -16.1

In [407]:
weathers_json = pd.read_json('data/historical_weather/{}_{}.json'.format(month, year))
weathers_json

temperature  wind_speed  weather_code  precip  \
2019-01-01 06:00:00            7          17           116     0.0   
2019-01-01 12:00:00            9          11           119     0.0   
2019-01-02 06:00:00            6           8           116     0.0   
2019-01-02 12:00:00            8          14           119     0.0   
2019-01-03 06:00:00            4           7           116     0.0   
2019-01-03 12:00:00            7          13           176     0.1   
2019-01-04 06:00:00            4          10           113     0.0   
2019-01-04 12:00:00            6           8           113     0.0   
2019-01-05 06:00:00            5          14           116     0.0   
2019-01-05 12:00:00            8          15           116     0.0   
2019-01-06 06:00:00            7          10           122     0.0   
2019-01-06 12:00:00            9           9           176     0.1   
2019-01-07 06:00:00           10          21           122     0.0   
2019-01-07 12:00:00           10          29           119     0.0   
2019-01-08 06:00:00            6          23           116     0.0   
2019-01-08 12:00:00            7          20           116     0.0   
2019-01-10 06:00:00            8           9           143     0.1   
2019-01-10 12:00:00            9          12           266     0.2   
2019-01-11 06:00:00            7          18           176     0.1   
2019-01-11 12:00:00            9          14           116     0.0   
2019-01-12 06:00:00            9          29           122     0.0   
2019-01-12 12:00:00            9          29           122     0.0   
2019-01-13 06:00:00            9          37           122     0.0   
2019-01-13 12:00:00           12          33           122     0.0   
2019-01-14 06:00:00            6          23           116     0.0   
2019-01-14 12:00:00            8          21           116     0.0   
2019-01-16 06:00:00            7          25           263     0.5   
2019-01-16 12:00:00            7          27           116     0.0   
2019-01-18 06:00:00            6          36           296     2.4   
2019-01-18 12:00:00            9          16           176     0.8   
2019-01-19 06:00:00            6           5           119     0.0   
2019-01-19 12:00:00            7           5           122     0.0   
2019-01-20 06:00:00            5          10           116     0.0   
2019-01-20 12:00:00            7          31           266     0.6   
2019-01-21 06:00:00            2          10           116     0.0   
2019-01-21 12:00:00            6          26           122     0.0   
2019-01-22 06:00:00            3          27           116     0.0   
2019-01-22 12:00:00            5          28           116     0.0   
2019-01-23 06:00:00            2          18           116     0.0   
2019-01-23 12:00:00            5          14           116     0.0   
2019-01-24 06:00:00            5          12           122     0.0   
2019-01-24 12:00:00            8           9           302     2.9   
2019-01-25 06:00:00            9          24           119     0.0   
2019-01-25 12:00:00           11          26           353     0.2   
2019-01-26 06:00:00            9          24           119     0.0   
2019-01-26 12:00:00           10          26           296     1.9   
2019-01-27 06:00:00            4          44           116     0.0   
2019-01-27 12:00:00            5          43           116     0.0   
2019-01-28 06:00:00            2          14           116     0.0   
2019-01-28 12:00:00            5          12           119     0.0   
2019-01-29 06:00:00            4          21           296     1.1   
2019-01-29 12:00:00            4          33           230     4.3   
2019-01-30 06:00:00            1          20           116     0.0   
2019-01-30 12:00:00            4          18           116     0.0   
2019-01-31 06:00:00            3          21           122     0.0   
2019-01-31 12:00:00            4          36           317     0.8   

                       MM/DD/Y

In [408]:
class Weather:
    def __init__(self, date, temp, wind, precip):
        self.date = date
        self.weather_m = 'N/A'
        self.weather_a = 'N/A'
        self.temp = temp
        self.wind = wind
        self.precip = precip
        
    def __iter__(self):
        yield self.date
        yield self.weather_m
        yield self.weather_a
        yield self.temp
        yield self.wind
        yield self.precip
        
    @classmethod
    def columns(cls):
        return['date', 'weather_m', 'weather_a', 'temp', 'wind', 'precip']
        

In [409]:
class AllWeather:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        self.arr[item.date] = item
        
    def get_item(self, date):
        if date in self.arr:
            return self.arr[date]
        else:
            return False
    
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Weather.columns())

In [410]:
weathers = AllWeather()

for datetime in weathers_json.index:
    date = weathers_json.loc[datetime, 'MM/DD/YY']
    temp = weathers_json.loc[datetime, 'temperature']
    wind = weathers_json.loc[datetime, 'wind_speed']
    precip = weathers_json.loc[datetime, 'precip']
    hour = weathers_json.loc[datetime, 'hour']
    weather_code = weathers_json.loc[datetime, 'weather_code']
    weather = weathers.get_item(date)
    if weather == False:
        weather = Weather(date, temp, wind, precip)
        weather.weather_m = weather_code
        weathers.set_item(weather)
    else:
        weather.weather_a = weather_code

In [411]:
weathers.items

date  weather_m  weather_a  temp  wind  precip
0   01/01/2019        116        119     7    17     0.0
1   01/02/2019        116        119     6     8     0.0
2   01/03/2019        116        176     4     7     0.0
3   01/04/2019        113        113     4    10     0.0
4   01/05/2019        116        116     5    14     0.0
5   01/06/2019        122        176     7    10     0.0
6   01/07/2019        122        119    10    21     0.0
7   01/08/2019        116        116     6    23     0.0
8   01/10/2019        143        266     8     9     0.1
9   01/11/2019        176        116     7    18     0.1
10  01/12/2019        122        122     9    29     0.0
11  01/13/2019        122        122     9    37     0.0
12  01/14/2019        116        116     6    23     0.0
13  01/16/2019        263        116     7    25     0.5
14  01/18/2019        296        176     6    36     2.4
15  01/19/2019        119        122     6     5     0.0
16  01/20/2019        116        266     5    10     0.0
17  01/21/2019        116        122     2    10     0.0
18  01/22/2019        116        116     3    27     0.0
19  01/23/2019        116        116     2    18     0.0
20  01/24/2019        122        302     5    12     0.0
21  01/25/2019        119        353     9    24     0.0
22  01/26/2019        119        296     9    24     0.0
23  01/27/2019        116        116     4    44     0.0
24  01/28/2019        116        119     2    14     0.0
25  01/29/2019        296        230     4    21     1.1
26  01/30/2019        116        116     1    20     0.0
27  01/31/2019        122        317     3    21     0.0

In [412]:
### Now I need to create Calendar object and create dayofweek etc
### so that I can add percentile growth into sales data

class Date:
    def __init__(self, date):
        self.date = date
        self.dayofweek = pd.Timestamp(pd.to_datetime(date)).dayofweek
        self.weeknum = pd.to_datetime(date).strftime("%W")
        self.day = pd.to_datetime(date).day
        self.month = pd.to_datetime(date).month
        self.year = pd.to_datetime(date).year
        self.holiday = 0
        
    def __iter__(self):
        yield self.dayofweek
        yield self.weeknum
        yield self.day
        yield self.month
        yield self.year
        yield self.holiday
        
    @classmethod
    def columns(cls):
        return['dayofweek', 'weeknum', 'day', 'month', 'year', 'holiday']
        
class Calendar:
    def __init__(self):
        self.arr = {}
        
    def set_date(self, item):
        self.arr[item.date] = item
        
    def get_date(self, date):
        if date in self.arr:
            return self.arr[date]
        else:
            return False
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Day.columns())

In [413]:
class Product:
    __lastid = 1
    
    def __init__(self, name, category):
        self.id = Product.__lastid
        self.name = name
        self.category = category
        
        Product.__lastid += 1
        
    def __iter__(self):
        yield self.id
        yield self.name
        yield self.category
    
    @classmethod
    def columns(cls):
        return ['id', 'name', 'category']
        
    
class Sales:
    def __init__(self, soldnum, date=None, product=None, weather=None, growth=None):
        self.product = product
        self.date = date
        self.soldnum = soldnum
        self.prevdaysales = []
        self.weather = weather
        self.growth = growth
        
    def set_prevdaysales(self, quantity):
        self.prevdaysales.append(quantity)
        
    def __iter__(self):
        yield self.product.id
        yield self.product.name
        yield self.product.category
        yield self.date.date
        yield self.date.dayofweek
        yield self.date.weeknum
        yield self.date.year
        yield self.date.holiday
        yield self.soldnum
        for i in range(7):
            try:
                prevsales = self.prevdaysales[i]
            except:
                prevsales = 'N/A'
            yield prevsales
            
        yield self.weather.weather_m
        yield self.weather.weather_a
        yield self.weather.temp
        yield self.weather.wind
        yield self.weather.precip
        
    @classmethod
    def columns(cls):
        return['id', 'name', 'category', 'date', 'dayofweek', 'weeknum', 'year',
               'holiday', 'soldnum', 'prevdaysales1', 'prevdaysales2', 'prevdaysales3', 
               'prevdaysales4', 'prevdaysales5', 'prevdaysales6', 'prevdaysales7',
               'weather_morning', 'weather_afternoon', 'temp', 'wind', 'precip']

In [414]:
class AllProduct:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        self.arr[item.name] = item
        
    def get_item(self, name):
        if name in self.arr:
            return self.arr[name]
        else:
            return False
        
    def delete(self, name):
        del self.arr[name]
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Product.columns())

In [430]:
class Trainingset:
    def __init__(self):
        self.arr = {}
        
    def set_item(self, item):
        key = '{}, {}'.format(item.date.date, item.product.name)
        self.arr[key] = item
        
    def get_item(self, name, date):
        key = '{}, {}'.format(date, name)
        if key in self.arr:
            return self.arr[key]
        else:
            return False
    
    def delete(self, name, date):
        key = '{}, {}'.format(date, name)
        del self.arr[key]
        
    @property
    def items(self):
        return pd.DataFrame(self.arr.values(), columns=Sales.columns())

In [439]:
trset = Trainingset()
products = AllProduct()
calendar = Calendar()
dt = None

# if all categories are True, that means the shop
# sold none of products, which means the shop were closed
closed = {'bakedgoods': False, 'lunch': False, 'breakfast': False} 

for date in dailysales_df.columns:
    dt = pd.to_datetime(date).strftime("%m/%d/%Y")
    calendar.set_date(Date(dt))
    
    for category in dailysales_df.index:
        sales = dailysales_df.loc[category, date]
        if sales == {}:
            closed[category] = True
            
        for itemname, sales in sales.items():
            product = products.get_item(itemname)
            weather = weathers.get_item(dt)
            
            if product == False:
                product = Product(itemname, category)
                products.set_item(product)
            
            if weather == False:
                weather = Weather(dt, 'N/A', 'N/A', 'N/A')
                weathers.set_item(weather)
            
            dateinfo = calendar.get_date(dt)
            
            growth = growths.get_item(product.category, dateinfo.dayofweek)
            sales = Sales(sales, dateinfo, product, weather, growth)
            trset.set_item(sales)

    if closed['bakedgoods'] and closed['lunch'] and closed['breakfast']:
        # if shops seems closed, do nothing
        closed['bakedgoods'] = False
        closed['lunch'] = False
        closed['breakfast'] = False
        continue
    else:
        # if the shop is open but seems the product sales is 0,
        # add the sales 0 into the training data
        for index in products.items.index:
            itemname = products.items.loc[index, 'name']
            item = trset.get_item(itemname, dt)
            if item is False:
                dateinfo = calendar.get_date(dt)
                product = products.get_item(itemname)
                weather = weathers.get_item(dt)
                
                if weather == False:
                    weather = Weather(dt, 'N/A', 'N/A', 'N/A')
                    weathers.set_item(weather)
                    
                growth = growths.get_item(product.category, dateinfo.dayofweek)
                sales = Sales(0, dateinfo, product, weather, growth)
                trset.set_item(sales)

In [440]:
trset.items

id                      name    category        date  dayofweek weeknum  \
0    125         Mixed Berry Scone  bakedgoods  01/02/2019          2      00   
1    126          Pain au Chocolat  bakedgoods  01/02/2019          2      00   
2    127                 Croissant  bakedgoods  01/02/2019          2      00   
3    128   Oatmeal & Raisin Cookie  bakedgoods  01/02/2019          2      00   
4    129              Custard Tart  bakedgoods  01/02/2019          2      00   
..   ...                       ...         ...         ...        ...     ...   
834  150  Mediterranean Vegan Wrap       lunch  01/31/2019          3      04   
835  151          Cheese Croissant   breakfast  01/31/2019          3      04   
836  152                   Cruffin  bakedgoods  01/31/2019          3      04   
837  153      Chicken Serrano Roll       lunch  01/31/2019          3      04   
838  154           Chicken Serrano       lunch  01/31/2019          3      04   

     year  holiday  soldnum prevdaysales1  ... prevdaysales3 prevdaysales4  \
0    2019        0       17           N/A  ...           N/A           N/A   
1    2019        0        5           N/A  ...           N/A           N/A   
2    2019        0        5           N/A  ...           N/A           N/A   
3    2019        0        3           N/A  ...           N/A           N/A   
4    2019        0        4           N/A  ...           N/A           N/A   
..    ...      ...      ...           ...  ...           ...           ...   
834  2019        0        0           N/A  ...           N/A           N/A   
835  2019        0        0           N/A  ...           N/A           N/A   
836  2019        0        0           N/A  ...           N/A           N/A   
837  2019        0        0           N/A  ...           N/A           N/A   
838  2019        0        0           N/A  ...           N/A           N/A   

    prevdaysales5 prevdaysales6 prevdaysales7 weather_morning  \
0             N/A           N/A           N/A             116   
1             N/A           N/A           N/A             116   
2             N/A           N/A           N/A             116   
3             N/A           N/A           N/A             116   
4             N/A           N/A           N/A             116   
..            ...           ...           ...             ...   
834           N/A           N/A           N/A             122   
835           N/A           N/A           N/A             122   
836           N/A           N/A           N/A             122   
837           N/A           N/A           N/A             122   
838           N/A           N/A           N/A             122   

    weather_afternoon temp wind precip  
0                 119    6    8      0  
1                 119    6    8      0  
2                 119    6    8      0  
3                 119    6    8      0  
4                 119    6    8      0  
..                ...  ...  ...    ...  
834               317    3   21      0  
835               317    3   21      0  
836               317    3   21      0  
837               317    3   21      0  
838               317    3   21      0  

[839 rows x 21 columns]

In [441]:
products.items

id                      name    category
0   125         Mixed Berry Scone  bakedgoods
1   126          Pain au Chocolat  bakedgoods
2   127                 Croissant  bakedgoods
3   128   Oatmeal & Raisin Cookie  bakedgoods
4   129              Custard Tart  bakedgoods
5   130                  Cinnabun  bakedgoods
6   131             Anzac Biscuit  bakedgoods
7   132       Snickers Boost Ball  bakedgoods
8   133          Almond Financier  bakedgoods
9   134          Coconut Snowball  bakedgoods
10  135            Pain au Raisin  bakedgoods
11  136     Peanut Butter Brownie  bakedgoods
12  137          Wholemeal Muffin  bakedgoods
13  138         Festive Financier  bakedgoods
14  139           Sourdough Toast   breakfast
15  140              Sausage Roll   breakfast
16  141          Organic Porridge   breakfast
17  142      Ham Cheese Croissant   breakfast
18  143   Berry & Granola Parfait   breakfast
19  144                   Caprese       lunch
20  145           Turkey Sandwich       lunch
21  146        Chicken Bacon Wrap       lunch
22  147          Chilli Tuna Wrap       lunch
23  148      Ham & Cheese Toastie       lunch
24  149           Soup of the Day       lunch
25  150  Mediterranean Vegan Wrap       lunch
26  151          Cheese Croissant   breakfast
27  152                   Cruffin  bakedgoods
28  153      Chicken Serrano Roll       lunch
29  154           Chicken Serrano       lunch
30  155                Vegan Wrap       lunch

In [442]:
from datetime import datetime

In [443]:
count = 0
targetdate = None

for index in trset.items.index:
    currentitem = trset.get_item(trset.items.loc[index, 'name'], trset.items.loc[index, 'date'])
    currentdt = pd.to_datetime(currentitem.date.date)
    name = currentitem.product.name
    
    while len(currentitem.prevdaysales) < 7:
        if count < -20:
            break
        count -= 1
        
        prevdt = currentdt + timedelta(days=count)
        prevdt = prevdt.strftime("%m/%d/%Y")
        previtem = trset.get_item(name, prevdt)
        if previtem is not False:
            currentitem.set_prevdaysales(previtem.soldnum)
        else:
            continue
    count = 0

In [444]:
trset.items

id                      name    category        date  dayofweek weeknum  \
0    125         Mixed Berry Scone  bakedgoods  01/02/2019          2      00   
1    126          Pain au Chocolat  bakedgoods  01/02/2019          2      00   
2    127                 Croissant  bakedgoods  01/02/2019          2      00   
3    128   Oatmeal & Raisin Cookie  bakedgoods  01/02/2019          2      00   
4    129              Custard Tart  bakedgoods  01/02/2019          2      00   
..   ...                       ...         ...         ...        ...     ...   
834  150  Mediterranean Vegan Wrap       lunch  01/31/2019          3      04   
835  151          Cheese Croissant   breakfast  01/31/2019          3      04   
836  152                   Cruffin  bakedgoods  01/31/2019          3      04   
837  153      Chicken Serrano Roll       lunch  01/31/2019          3      04   
838  154           Chicken Serrano       lunch  01/31/2019          3      04   

     year  holiday  soldnum prevdaysales1  ... prevdaysales3 prevdaysales4  \
0    2019        0       17           N/A  ...           N/A           N/A   
1    2019        0        5           N/A  ...           N/A           N/A   
2    2019        0        5           N/A  ...           N/A           N/A   
3    2019        0        3           N/A  ...           N/A           N/A   
4    2019        0        4           N/A  ...           N/A           N/A   
..    ...      ...      ...           ...  ...           ...           ...   
834  2019        0        0             2  ...             1             0   
835  2019        0        0             0  ...             0             0   
836  2019        0        0             0  ...             0             0   
837  2019        0        0             0  ...             0             0   
838  2019        0        0             0  ...             0             0   

    prevdaysales5 prevdaysales6 prevdaysales7 weather_morning  \
0             N/A           N/A           N/A             116   
1             N/A           N/A           N/A             116   
2             N/A           N/A           N/A             116   
3             N/A           N/A           N/A             116   
4             N/A           N/A           N/A             116   
..            ...           ...           ...             ...   
834             2             2             2             122   
835             0             0             0             122   
836             0             0             0             122   
837             0             0             0             122   
838             0             0             0             122   

    weather_afternoon temp wind precip  
0                 119    6    8      0  
1                 119    6    8      0  
2                 119    6    8      0  
3                 119    6    8      0  
4                 119    6    8      0  
..                ...  ...  ...    ...  
834               317    3   21      0  
835               317    3   21      0  
836               317    3   21      0  
837               317    3   21      0  
838               317    3   21      0  

[839 rows x 21 columns]

In [463]:
# train = {key:val for key, val in trset.items.prevdaysales if val != 42}
train = Trainingset()

for key in trset.arr:
    item = trset.arr[key]
    if item.prevdaysales == [0, 0, 0, 0, 0, 0, 0]:
        continue
    else:
        train.set_item(item)
#     item = trset.items.loc[index]
#     if item.prevdaysales is not [0, 0, 0, 0, 0, 0, 0]:
#         train.set_item(item)

In [464]:
train.items

id                      name    category        date  dayofweek weeknum  \
0    125         Mixed Berry Scone  bakedgoods  01/02/2019          2      00   
1    126          Pain au Chocolat  bakedgoods  01/02/2019          2      00   
2    127                 Croissant  bakedgoods  01/02/2019          2      00   
3    128   Oatmeal & Raisin Cookie  bakedgoods  01/02/2019          2      00   
4    129              Custard Tart  bakedgoods  01/02/2019          2      00   
..   ...                       ...         ...         ...        ...     ...   
770  129              Custard Tart  bakedgoods  01/31/2019          3      04   
771  136     Peanut Butter Brownie  bakedgoods  01/31/2019          3      04   
772  142      Ham Cheese Croissant   breakfast  01/31/2019          3      04   
773  147          Chilli Tuna Wrap       lunch  01/31/2019          3      04   
774  150  Mediterranean Vegan Wrap       lunch  01/31/2019          3      04   

     year  holiday  soldnum prevdaysales1  ... prevdaysales3 prevdaysales4  \
0    2019        0       17           N/A  ...           N/A           N/A   
1    2019        0        5           N/A  ...           N/A           N/A   
2    2019        0        5           N/A  ...           N/A           N/A   
3    2019        0        3           N/A  ...           N/A           N/A   
4    2019        0        4           N/A  ...           N/A           N/A   
..    ...      ...      ...           ...  ...           ...           ...   
770  2019        0        0             6  ...             3             5   
771  2019        0        0             0  ...             2             0   
772  2019        0        0             3  ...             5             3   
773  2019        0        0             1  ...             1             0   
774  2019        0        0             2  ...             1             0   

    prevdaysales5 prevdaysales6 prevdaysales7 weather_morning  \
0             N/A           N/A           N/A             116   
1             N/A           N/A           N/A             116   
2             N/A           N/A           N/A             116   
3             N/A           N/A           N/A             116   
4             N/A           N/A           N/A             116   
..            ...           ...           ...             ...   
770             7             8             4             122   
771             0             3             2             122   
772             2             4             6             122   
773             0             0             1             122   
774             2             2             2             122   

    weather_afternoon temp wind precip  
0                 119    6    8      0  
1                 119    6    8      0  
2                 119    6    8      0  
3                 119    6    8      0  
4                 119    6    8      0  
..                ...  ...  ...    ...  
770               317    3   21      0  
771               317    3   21      0  
772               317    3   21      0  
773               317    3   21      0  
774               317    3   21      0  

[775 rows x 21 columns]

In [465]:
train.items.loc[500]

id                               145
name                 Turkey Sandwich
category                       lunch
date                      01/21/2019
dayofweek                          0
weeknum                           03
year                            2019
holiday                            0
soldnum                            3
prevdaysales1                      0
prevdaysales2                      2
prevdaysales3                      2
prevdaysales4                      3
prevdaysales5                      2
prevdaysales6                      1
prevdaysales7                      0
weather_morning                  116
weather_afternoon                122
temp                               2
wind                              10
precip                             0
Name: 500, dtype: object

In [466]:
trset.items.loc[trset.items['date'] == '01/10/2019']

id                      name    category        date  dayofweek weeknum  \
206  125         Mixed Berry Scone  bakedgoods  01/10/2019          3      01   
207  130                  Cinnabun  bakedgoods  01/10/2019          3      01   
208  126          Pain au Chocolat  bakedgoods  01/10/2019          3      01   
209  128   Oatmeal & Raisin Cookie  bakedgoods  01/10/2019          3      01   
210  127                 Croissant  bakedgoods  01/10/2019          3      01   
211  129              Custard Tart  bakedgoods  01/10/2019          3      01   
212  136     Peanut Butter Brownie  bakedgoods  01/10/2019          3      01   
213  131             Anzac Biscuit  bakedgoods  01/10/2019          3      01   
214  132       Snickers Boost Ball  bakedgoods  01/10/2019          3      01   
215  135            Pain au Raisin  bakedgoods  01/10/2019          3      01   
216  134          Coconut Snowball  bakedgoods  01/10/2019          3      01   
217  137          Wholemeal Muffin  bakedgoods  01/10/2019          3      01   
218  139           Sourdough Toast   breakfast  01/10/2019          3      01   
219  140              Sausage Roll   breakfast  01/10/2019          3      01   
220  141          Organic Porridge   breakfast  01/10/2019          3      01   
221  142      Ham Cheese Croissant   breakfast  01/10/2019          3      01   
222  143   Berry & Granola Parfait   breakfast  01/10/2019          3      01   
223  144                   Caprese       lunch  01/10/2019          3      01   
224  150  Mediterranean Vegan Wrap       lunch  01/10/2019          3      01   
225  148      Ham & Cheese Toastie       lunch  01/10/2019          3      01   
226  145           Turkey Sandwich       lunch  01/10/2019          3      01   
227  146        Chicken Bacon Wrap       lunch  01/10/2019          3      01   
228  149           Soup of the Day       lunch  01/10/2019          3      01   
229  133          Almond Financier  bakedgoods  01/10/2019          3      01   
230  138         Festive Financier  bakedgoods  01/10/2019          3      01   
231  147          Chilli Tuna Wrap       lunch  01/10/2019          3      01   

     year  holiday  soldnum prevdaysales1  ... prevdaysales3 prevdaysales4  \
206  2019        0       21            15  ...            15            12   
207  2019        0        6             7  ...             3             5   
208  2019        0        4             3  ...             4             0   
209  2019        0       15             3  ...             3             2   
210  2019        0        6             6  ...             4             2   
211  2019        0        6             6  ...             3             5   
212  2019        0        1             2  ...             1             0   
213  2019        0       17             9  ...             5             2   
214  2019        0        7             6  ...             4             2   
215  2019        0        5             0  ...             4             6   
216  2019        0        1             2  ...             2             1   
217  2019        0        3             6  ...             2             3   
218  2019        0        6             4  ...             2             0   
219  2019        0        7             4  ...             3             9   
220  2019        0        4             5  ...             5             0   
221  2019        0        3             2  ...             5             4   
222  2019        0        5             1  ...             3             0   
223  2019        0        2             3  ...             2             0   
224  2019        0        2             2  ...             0             0   
225  2019        0        3             3  ...             0             0   
226  2019        0        1             2  ...             2             0   
227  2019        0        3             3  ...             4             0   
228  2019        0      